https://github.com/monologg/KoELECTRA?tab=readme-ov-file

# 0) 모듈 실행

##기본 모듈

In [1]:
# Matplotlib 와 pandas 그래프에서 한글 사용 설정
import sys

## 노트북이 코랩에서 실행중인지 체크
if 'google.colab' in sys.modules:
  !echo 'debconf debconf/frontend select Noninterative' | debconf-set-selections
  ## 나눔 폰트 설치
  !sudo apt-get -qq -y install fonts-nanum
  import matplotlib.font_manager as fm
  font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
  for fpath in font_files:
    fm.fontManager.addfont(fpath)

  # 나눔바른고딕 폰트 설정
  import matplotlib.pyplot as plt
  plt.rc('font', family='NanumBarunGothic')
  # 마이너스 기호 표시 오류 수정
  import matplotlib
  matplotlib.rcParams['axes.unicode_minus'] = False


debconf: unable to initialize frontend: Noninterative
debconf: (Can't locate Debconf/FrontEnd/Noninterative.pm in @INC (you may need to install the Debconf::FrontEnd::Noninterative module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.34.0 /usr/local/share/perl/5.34.0 /usr/lib/x86_64-linux-gnu/perl5/5.34 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.34 /usr/share/perl/5.34 /usr/local/lib/site_perl) at (eval 17) line 2, <> line 1.)
debconf: falling back to frontend: Noninteractive
Selecting previously unselected package fonts-nanum.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 8.6 MB/s eta 0:00:00


In [3]:
!apt-get update -qq

In [4]:
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum-coding.
(Reading database ... 121941 files and directories currently installed.)
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-coding (2.5-3) ...
Setting up fonts-nanum-eco (1.000-7) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [5]:
import matplotlib.font_manager as fm
sys_font = fm.findSystemFonts()

# 한글 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
#fm._rebuild()

NanumBarunGothic


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import random
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from tqdm import tqdm
import torch.nn.functional as F

In [8]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

## 모델링

In [9]:
# GPU 할당 확인
!nvidia-smi

Wed May 22 05:19:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
!pip install transformers pytorch_lightning tokenizers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 14.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [11]:
import tqdm
import torch
import tensorflow as tf

from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_scheduler
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [12]:
from transformers import AutoTokenizer
from transformers import ElectraForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from torch.nn import functional as F
from tqdm.notebook import tqdm

# 1) 데이터 가져오기

## 감성대화 말뭉치
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86
- 일반인 1,500명 대상의 인터뷰 및 크라우드 소싱 수행
- 우울증 환자 대상 WOZ 대화 수집

=> 전처리 과정으로 "한사람만의 대화를 줄글로 변형 - 질문 제거"

### 1. 데이터 가져오기

In [16]:
train = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.xlsx")
test = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.xlsx")

In [17]:
train_df = train.copy()
test_df = test.copy()

### 2. 전처리

In [18]:
# "사람문장3" 결측값 대체
train_df.fillna('',inplace=True)
test_df.fillna('',inplace=True)

# 문장 병합
train_df['사람문장'] = train_df['사람문장1']+ " " + train_df['사람문장2']+ " " +train_df['사람문장3']
test_df['사람문장'] = test_df['사람문장1']+ " " +test_df['사람문장2']+ " " +test_df['사람문장3']

# "감정 분류"과 "문장"만 추출
train_df = train_df[['사람문장', '감정_대분류']]
test_df = test_df[['사람문장', '감정_대분류']]

# 컬럼명 변경
train_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)
test_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)

# label encoding
le = LabelEncoder()
le=le.fit(train_df['sentiment'])
train_df['target']=le.transform(train_df['sentiment'])

le=le.fit(test_df['sentiment'])
test_df['target']=le.transform(test_df['sentiment'])

# text, target
train_conv = train_df[['text','target']]
test_conv = test_df[['text','target']]

<ipython-input-18-1d7e370bec27>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.rename(columns={'사람문장': 'text',
<ipython-input-18-1d7e370bec27>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.rename(columns={'사람문장': 'text',
<ipython-input-18-1d7e370bec27>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['target']=le.transform(train_df['sentiment'])


In [19]:
# 라벨 인코딩 결과 확인
sentiment = train_df[['sentiment', 'target']].drop_duplicates(subset=['sentiment']) # sentiment 컬럼에 대해 중복된 값을 갖고 있는 열 제거
df_unique = sentiment.sort_values(by='target', ascending=True)
df_unique = df_unique.reset_index(drop=True)
df_unique

,sentiment,target
0,기쁨,0
1,당황,1
2,분노,2
3,불안,3
4,상처,4
5,슬픔,5


# 2) 데이터 증강
데이터 불균형으로 인해 분류 전에 전처리하는 방식인 smote 택함

**SMOTE(= Synthetic Minority Over-sampling Technique)**

SMOTE는 합성을 기반으로 하는 방법으로, minority class의 샘플을 가져와 이들을 잇는 선에서 중간값을 택하여 만들어내는 방식

Train

In [20]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = train_conv.text.to_numpy().reshape(-1, 1)
target = train_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
train_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
train_conv2

,text,target
0,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,2
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,2
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,2
...,...,...
55915,여자 친구 부모님이 하시던 사업이 망하고 여자 친구는 내게 이별을 통보했어. 서로의...,5
55916,아이들은 아직 한참 어린데 회사 생활은 위태롭고 하니 많이 우울해져. 요즘 코로나 ...,5
55917,미대 입시를 준비하고 있는 나보다 친구가 대충 그린 그림이 더 나아서 낙담했어. 내...,5
55918,실수를 조금 했다고 회사에서 상사가 호통을 쳐서 어찌나 마음이 씁쓸했는지. 난 최선...,5


Test

In [21]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = test_conv.text.to_numpy().reshape(-1, 1)
target = test_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
test_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
test_conv2

,text,target
0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,3
1,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워. 나에게 너무 크...,3
2,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워. 무섭게 생겼는데도 업무를 보려...,3
3,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다. 첫 직장...,3
4,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼. 내가 낯가림이 심해서 친해질 수...,3
...,...,...
7537,지금 많이 속상한 일이 있는데 같이 술 한 잔 마실 만한 친구가 없어. 내 주변에는...,5
7538,심장마비로 남편을 떠나보내고 매일 아침이면 눈을 뜨기가 두려워. 차갑게 식어버렸던 ...,5
7539,지금 많이 속상한 일이 있는데 같이 술 한 잔 마실 만한 친구가 없어. 내 주변에는...,5
7540,어릴 적 술에 취해 우리 가족을 학대했던 아버지가 간 이식을 해달라며 오랜만에 찾아...,5


# 3) 데이터 전처리
모델 적합을 위해 데이터가 적절히 전처리되어야 하며 이때 전처리는 토큰화, 정수 인코딩, 패딩을 의미함

In [22]:
import re
# 텍스트 클린징 함수 정의
def clean_text(text):
    # 특수문자 제거
    text = re.sub(r'[^가-힣A-Za-z0-9 ]', '', text)

    # 여러 개의 공백은 하나로 줄이기
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [23]:
train_df = train_conv2.copy()
test_df = test_conv2.copy()

train_df['text'] = train_df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)

In [24]:
train_df

,text,target
0,일은 왜 해도 해도 끝이 없을까 화가 난다 그냥 내가 해결하는 게 나아 남들한테 부...,2
1,이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 최...,2
2,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 섭...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사인 나에게 먼...,2
...,...,...
55915,여자 친구 부모님이 하시던 사업이 망하고 여자 친구는 내게 이별을 통보했어 서로의 ...,5
55916,아이들은 아직 한참 어린데 회사 생활은 위태롭고 하니 많이 우울해져 요즘 코로나 때...,5
55917,미대 입시를 준비하고 있는 나보다 친구가 대충 그린 그림이 더 나아서 낙담했어 내 ...,5
55918,실수를 조금 했다고 회사에서 상사가 호통을 쳐서 어찌나 마음이 씁쓸했는지 난 최선을...,5


토큰화
- KoElectra의 토크나이저를 쓰기 위해선 from_pretrained 함수의 인자로 monologg/koelectra-small-v3-discriminator 를 넣음

- ##이 같은 것들은 BERT나 ELECTRA 같은 토크나이저의 특징 중 하나
- 단어를 더 잘 이해할 수 있도록 도움

In [25]:
# 데이터셋 클래스: SentimentReviewDataset 클래스를 사용하여 입력 데이터를 준비
from transformers import ElectraTokenizer
from torch.utils.data import DataLoader, Dataset

class SentimentReviewDataset(Dataset):

  def __init__(self, dataset):
    self.dataset = dataset
    self.tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, ].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt', #pytorch 사용
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [26]:
train_dataset = SentimentReviewDataset(train_df)
test_dataset = SentimentReviewDataset(test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

In [27]:
train_dataset[2]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(tensor([    2,  6387,  4073, 10390,  4007, 10199, 18781, 14643,  4070,  2041,
          4525,  4228,  6269,  3093,  4110,  7465,  8070,  6556,  6243, 20653,
          4149,  8692, 29895,  3258,  3081,  2639,  4034,  6226,  4007,  4294,
         14419,  3258,  8730,  4034,  2048,  4275,  4071,  6681,  4112,  6713,
          4110, 29252, 10766,  4110, 10855,  4129,  8028,  4034,  2054,  2376,
          8692,  4736,  4292,  2048,  2024,  4153,     3,     0,     0,     0,
             0,     0,     0,     0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]),
 2)

# 4) 모델 생성

In [37]:
# GPU 사용
device = torch.device("cuda")

In [38]:
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
#model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
'''from transformers import ElectraModel, ElectraTokenizer

model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")'''

'from transformers import ElectraModel, ElectraTokenizer\n\nmodel = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")\ntokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")'

In [40]:
# 한번 실행해보기
text, attention_mask, y = train_dataset[0]
model(text.unsqueeze(0), attention_mask = attention_mask.unsqueeze(0))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [41]:
# 분류 헤드 수정 - KoElectra 모델의 출력층을 6개의 클래스로 분류할 수 있도록 수정
import torch.nn as nn

model.classifier.out_proj = nn.Linear(in_features=768, out_features=6)

In [42]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [31]:
# 레이블 값 확인 : 레이블 값이 올바른 범위 내에 있는지 확인
print(y_batch.unique())
# 모델 출력 형태 확인
print(y_pred.shape)
print(y_batch.shape)

NameError: name 'y_batch' is not defined

# 5) 모델 학습

In [43]:
epochs = 5
batch_size = 16

optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [44]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [48]:
losses = []
accuracies = []

# 모델을 올바른 디바이스로 이동
model.to(device)

for i in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        # 배치 데이터를 GPU로 이동
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()

        # 모델 예측
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        y_pred = outputs.logits

        # 손실 계산
        loss = F.cross_entropy(y_pred, y_batch)

        # 역전파
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        total_loss += loss.item()
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        batches += 1
        if batches % 100 == 0:
            print("Batch Loss:", total_loss, "Accuracy:", correct / total)

    losses.append(total_loss)
    accuracies.append(correct / total)
    print(f"Epoch {i+1}, Train Loss: {total_loss}, Accuracy: {correct / total}")


  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 136.59110033512115 Accuracy: 0.460625
Batch Loss: 267.8924931883812 Accuracy: 0.4803125
Batch Loss: 395.01288414001465 Accuracy: 0.4847916666666667
Batch Loss: 515.8171733617783 Accuracy: 0.49765625
Batch Loss: 633.3232195973396 Accuracy: 0.51025
Batch Loss: 749.7831844687462 Accuracy: 0.5194791666666667
Batch Loss: 858.0040547847748 Accuracy: 0.5304464285714285
Batch Loss: 967.7677550911903 Accuracy: 0.537109375
Batch Loss: 1073.3118022680283 Accuracy: 0.5443055555555556
Batch Loss: 1179.697869837284 Accuracy: 0.5499375
Batch Loss: 1279.4284699559212 Accuracy: 0.5575
Batch Loss: 1380.44048255682 Accuracy: 0.5633854166666666
Batch Loss: 1481.7865645885468 Accuracy: 0.5686057692307692
Batch Loss: 1584.018971145153 Accuracy: 0.5736160714285714
Batch Loss: 1687.775984942913 Accuracy: 0.5773333333333334
Batch Loss: 1788.166271686554 Accuracy: 0.5805078125
Batch Loss: 1885.5675617158413 Accuracy: 0.5845588235294118
Batch Loss: 1979.8223159313202 Accuracy: 0.5888541666666667
Batc

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 85.53485330939293 Accuracy: 0.69
Batch Loss: 167.60844865441322 Accuracy: 0.7028125
Batch Loss: 248.07357740402222 Accuracy: 0.70625
Batch Loss: 330.0728804767132 Accuracy: 0.7071875
Batch Loss: 413.4576374590397 Accuracy: 0.706875
Batch Loss: 498.9330685585737 Accuracy: 0.7044791666666667
Batch Loss: 579.7346451431513 Accuracy: 0.7055357142857143
Batch Loss: 663.1624878197908 Accuracy: 0.7065625
Batch Loss: 747.643506899476 Accuracy: 0.7052083333333333
Batch Loss: 825.7640255540609 Accuracy: 0.7063125
Batch Loss: 906.8792233020067 Accuracy: 0.706590909090909
Batch Loss: 989.6018254905939 Accuracy: 0.7066666666666667
Batch Loss: 1071.4431159347296 Accuracy: 0.706875
Batch Loss: 1153.2679135501385 Accuracy: 0.7071875
Batch Loss: 1231.2532895952463 Accuracy: 0.7079583333333334
Batch Loss: 1307.8420557528734 Accuracy: 0.70875
Batch Loss: 1391.9208246022463 Accuracy: 0.7087132352941177
Batch Loss: 1474.555416598916 Accuracy: 0.7079513888888889
Batch Loss: 1548.1043143719435 Acc

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 67.41336670517921 Accuracy: 0.765625
Batch Loss: 145.103778347373 Accuracy: 0.7484375
Batch Loss: 214.91106188297272 Accuracy: 0.749375
Batch Loss: 286.9853124320507 Accuracy: 0.74890625
Batch Loss: 359.48511427640915 Accuracy: 0.749375
Batch Loss: 434.93738198280334 Accuracy: 0.7439583333333334
Batch Loss: 505.390444368124 Accuracy: 0.7453571428571428
Batch Loss: 577.5953772068024 Accuracy: 0.74625
Batch Loss: 645.8998328000307 Accuracy: 0.7474305555555556
Batch Loss: 717.5109881013632 Accuracy: 0.7461875
Batch Loss: 784.7943807393312 Accuracy: 0.7480681818181818
Batch Loss: 861.3607220351696 Accuracy: 0.7467708333333334
Batch Loss: 931.191817060113 Accuracy: 0.7467788461538462
Batch Loss: 1002.009682893753 Accuracy: 0.7470535714285714
Batch Loss: 1071.69544570148 Accuracy: 0.7470833333333333
Batch Loss: 1146.1684481948614 Accuracy: 0.746484375
Batch Loss: 1213.4570579081774 Accuracy: 0.7470220588235295
Batch Loss: 1289.263461485505 Accuracy: 0.7459027777777778
Batch Loss:

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 59.51568886637688 Accuracy: 0.796875
Batch Loss: 122.9096337556839 Accuracy: 0.7903125
Batch Loss: 179.97353763878345 Accuracy: 0.79375
Batch Loss: 244.85105708241463 Accuracy: 0.78953125
Batch Loss: 307.6538926512003 Accuracy: 0.788375
Batch Loss: 368.79456619918346 Accuracy: 0.7866666666666666
Batch Loss: 432.5467105805874 Accuracy: 0.7860714285714285
Batch Loss: 495.642331674695 Accuracy: 0.785390625
Batch Loss: 558.0668934509158 Accuracy: 0.7854166666666667
Batch Loss: 621.6943776533008 Accuracy: 0.784375
Batch Loss: 683.6981242224574 Accuracy: 0.7847727272727273
Batch Loss: 746.7039034888148 Accuracy: 0.7846354166666667
Batch Loss: 811.5500031337142 Accuracy: 0.7836538461538461
Batch Loss: 871.1826164275408 Accuracy: 0.7845535714285714
Batch Loss: 931.0386695340276 Accuracy: 0.7852916666666667
Batch Loss: 996.6241316124797 Accuracy: 0.7848828125
Batch Loss: 1058.8874617144465 Accuracy: 0.7844852941176471
Batch Loss: 1119.4767783805728 Accuracy: 0.7846527777777778
Batch

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 53.89105190336704 Accuracy: 0.81375
Batch Loss: 106.88942573964596 Accuracy: 0.8175
Batch Loss: 161.3058329820633 Accuracy: 0.818125
Batch Loss: 211.97002658993006 Accuracy: 0.82171875
Batch Loss: 264.88413374871016 Accuracy: 0.822125
Batch Loss: 318.5700982734561 Accuracy: 0.8211458333333334
Batch Loss: 366.2898921966553 Accuracy: 0.8230357142857143
Batch Loss: 418.7927507162094 Accuracy: 0.823515625
Batch Loss: 473.2557148784399 Accuracy: 0.8233333333333334
Batch Loss: 524.4067590758204 Accuracy: 0.8231875
Batch Loss: 577.3198377043009 Accuracy: 0.8227272727272728
Batch Loss: 629.3315298557281 Accuracy: 0.8228645833333333
Batch Loss: 672.9037709794939 Accuracy: 0.8255288461538461
Batch Loss: 726.1331493034959 Accuracy: 0.8242410714285714
Batch Loss: 778.1001243814826 Accuracy: 0.8244583333333333
Batch Loss: 832.4810273051262 Accuracy: 0.82390625
Batch Loss: 883.6020162552595 Accuracy: 0.8237867647058823
Batch Loss: 936.8019005656242 Accuracy: 0.8235069444444445
Batch Loss

In [46]:
'''losses = []
accuracies = []

# 모델을 올바른 디바이스로 이동
model.to(device)

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
     # 배치 데이터를 GPU로 이동
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    # 손실 및 정확도 계산
    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)'''

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 178.53677594661713 Accuracy: tensor(0.1881, device='cuda:0')
Batch Loss: 344.23367869853973 Accuracy: tensor(0.2594, device='cuda:0')


KeyboardInterrupt: 

In [49]:
losses, accuracies

([3528.508292913437,
  2842.3084238916636,
  2507.989254295826,
  2181.0655539929867,
  1840.6551535502076],
 [0.6294527896995709,
  0.7112124463519314,
  0.7464949928469242,
  0.7830829756795422,
  0.8206366237482118])

In [50]:
# 모델 저장하기
model_path = "/content/drive/MyDrive/model_KoElectra_0522.pt"
torch.save(model.state_dict(), model_path)

# 5) 모델 평가

In [51]:
from sklearn.metrics import classification_report

In [52]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/472 [00:00<?, ?it/s]

Accuracy: tensor(0.7580, device='cuda:0')


# 5) 모델 추가 학습

- 저장된 모델을 불러온 후, 추가학습을 통해 모델 성능을 더 향상시킴
- 모델을 점진적으로 개선하고 데이터와 학습 조건을 조정하며 성능 최적화

In [53]:
model.classifier.out_proj = nn.Linear(in_features=768, out_features=6)
model.load_state_dict(torch.load(model_path))
model.to(device)

# 옵티마이저 설정
epochs = 5
batch_size = 32
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# 추가 학습 루프
for i in range(epochs):  # 추가 학습할 에포크 수
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        # 배치 데이터를 GPU로 이동
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()

        # 모델 예측
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        y_pred = outputs.logits

        # 손실 계산
        loss = F.cross_entropy(y_pred, y_batch)

        # 역전파
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        total_loss += loss.item()
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        batches += 1
        if batches % 100 == 0:
            print("Batch Loss:", total_loss, "Accuracy:", correct / total)

    print(f"Epoch {i+1}, Train Loss: {total_loss}, Accuracy: {correct / total}")

  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 41.832984022796154 Accuracy: 0.864375
Batch Loss: 84.1033119931817 Accuracy: 0.86140625
Batch Loss: 124.42772999405861 Accuracy: 0.8627083333333333
Batch Loss: 163.2092800885439 Accuracy: 0.86453125
Batch Loss: 205.6889376193285 Accuracy: 0.8626875
Batch Loss: 246.62286833673716 Accuracy: 0.86265625
Batch Loss: 285.36745547503233 Accuracy: 0.8637946428571428
Batch Loss: 325.8650929927826 Accuracy: 0.8640625
Batch Loss: 365.16887994110584 Accuracy: 0.8647569444444444
Batch Loss: 407.64009761065245 Accuracy: 0.8639375
Batch Loss: 446.96803487092257 Accuracy: 0.8639772727272728
Batch Loss: 489.10454250872135 Accuracy: 0.8640885416666667
Batch Loss: 531.9178154915571 Accuracy: 0.8632451923076923
Batch Loss: 575.8729309588671 Accuracy: 0.8624776785714285
Batch Loss: 617.9641757458448 Accuracy: 0.862375
Batch Loss: 660.7456681281328 Accuracy: 0.861875
Batch Loss: 702.585548967123 Accuracy: 0.8612683823529412
Epoch 1, Train Loss: 722.7687956392765, Accuracy: 0.8612660944206009


  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 36.1001121327281 Accuracy: 0.8809375
Batch Loss: 68.29496795684099 Accuracy: 0.886875
Batch Loss: 102.9073979780078 Accuracy: 0.8846875
Batch Loss: 136.919275470078 Accuracy: 0.885859375
Batch Loss: 171.47236323356628 Accuracy: 0.8855
Batch Loss: 204.9210446253419 Accuracy: 0.8861979166666667
Batch Loss: 238.59347301721573 Accuracy: 0.8870089285714285
Batch Loss: 270.662751480937 Accuracy: 0.8875390625
Batch Loss: 304.583153873682 Accuracy: 0.8878125
Batch Loss: 339.2127540744841 Accuracy: 0.88753125
Batch Loss: 374.42768044397235 Accuracy: 0.8872443181818181
Batch Loss: 408.96975714340806 Accuracy: 0.8872395833333333
Batch Loss: 444.6045195572078 Accuracy: 0.8869230769230769
Batch Loss: 477.14410733804107 Accuracy: 0.8873660714285714
Batch Loss: 510.86415066197515 Accuracy: 0.8876041666666666
Batch Loss: 545.7865035943687 Accuracy: 0.88759765625
Batch Loss: 581.2298336587846 Accuracy: 0.8871875
Epoch 2, Train Loss: 600.1621289588511, Accuracy: 0.8867668097281831


  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 26.942457403987646 Accuracy: 0.913125
Batch Loss: 52.59348772838712 Accuracy: 0.91390625
Batch Loss: 83.61121149361134 Accuracy: 0.9096875
Batch Loss: 111.33278165757656 Accuracy: 0.909921875
Batch Loss: 138.9256068840623 Accuracy: 0.9101875
Batch Loss: 168.93007188290358 Accuracy: 0.9086458333333334
Batch Loss: 194.88198424875736 Accuracy: 0.9099107142857142
Batch Loss: 223.54972904920578 Accuracy: 0.90953125
Batch Loss: 251.03270160779357 Accuracy: 0.9095486111111111
Batch Loss: 280.75268897786736 Accuracy: 0.9089375
Batch Loss: 307.67168794199824 Accuracy: 0.9089204545454546
Batch Loss: 335.95065308734775 Accuracy: 0.909375
Batch Loss: 363.74708246812224 Accuracy: 0.9098317307692307
Batch Loss: 393.14094764366746 Accuracy: 0.9094642857142857
Batch Loss: 422.7472458705306 Accuracy: 0.909125
Batch Loss: 449.65026984363794 Accuracy: 0.9092578125
Batch Loss: 478.2938425280154 Accuracy: 0.9092279411764705
Epoch 3, Train Loss: 492.13905815407634, Accuracy: 0.9090307582260372


  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 24.278346991166472 Accuracy: 0.9221875
Batch Loss: 46.66194260120392 Accuracy: 0.9240625
Batch Loss: 68.3587816953659 Accuracy: 0.9270833333333334
Batch Loss: 91.10878342017531 Accuracy: 0.926640625
Batch Loss: 112.79519275948405 Accuracy: 0.9271875
Batch Loss: 137.88040929473937 Accuracy: 0.9258854166666667
Batch Loss: 160.54697828553617 Accuracy: 0.9257589285714286
Batch Loss: 183.19250580482185 Accuracy: 0.925703125
Batch Loss: 207.66545327939093 Accuracy: 0.9256944444444445
Batch Loss: 231.35467561148107 Accuracy: 0.925
Batch Loss: 254.0629833806306 Accuracy: 0.9248295454545454
Batch Loss: 277.8057871479541 Accuracy: 0.92484375
Batch Loss: 299.89506204240024 Accuracy: 0.9253846153846154
Batch Loss: 324.1714274752885 Accuracy: 0.9251116071428571
Batch Loss: 348.2664297707379 Accuracy: 0.925
Batch Loss: 373.7229338847101 Accuracy: 0.9243359375
Batch Loss: 394.7736259698868 Accuracy: 0.9246323529411765
Epoch 4, Train Loss: 405.46358297020197, Accuracy: 0.9246781115879829


  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 18.781266290694475 Accuracy: 0.9415625
Batch Loss: 36.639161955565214 Accuracy: 0.9434375
Batch Loss: 55.175352942198515 Accuracy: 0.9413541666666667
Batch Loss: 73.95754129625857 Accuracy: 0.941484375
Batch Loss: 95.19358058832586 Accuracy: 0.9395625
Batch Loss: 112.83645394071937 Accuracy: 0.9403645833333333
Batch Loss: 131.22285430133343 Accuracy: 0.9404910714285715
Batch Loss: 149.4529950246215 Accuracy: 0.9404296875
Batch Loss: 170.30181474797428 Accuracy: 0.9399305555555556
Batch Loss: 190.65550846233964 Accuracy: 0.9395625
Batch Loss: 210.74389125034213 Accuracy: 0.9390056818181818
Batch Loss: 231.33503391966224 Accuracy: 0.9388541666666667
Batch Loss: 251.24419168569148 Accuracy: 0.9380528846153846
Batch Loss: 271.1015729922801 Accuracy: 0.9379017857142857
Batch Loss: 290.9413874093443 Accuracy: 0.93775
Batch Loss: 308.61872174590826 Accuracy: 0.9381640625
Batch Loss: 326.3561837533489 Accuracy: 0.9383823529411764
Epoch 5, Train Loss: 336.8205167138949, Accuracy: 0.

In [54]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/472 [00:00<?, ?it/s]

Accuracy: tensor(0.7717, device='cuda:0')
